In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
from re import sub
from decimal import Decimal


***


#### Cleaning up raw dataframe: erroneous 1st column name, missing year in Release Date, replace NaNs

In [2]:
df_raw = pd.read_csv('10_merged_movies_with_features_no_duplicates.csv')

# Add year for Release Date column
#df_raw['Release Date'] = [i+' '+yr for i in df_raw['Release Date']]

# Replace NaNs
df_raw['Budget'] = df_raw['Budget'].fillna('0')
df_raw['MPAA']   = df_raw['MPAA'].fillna('0')

In [3]:
#df_raw.head()
df_raw.columns

Index(['Unnamed: 0', 'Release', 'Calendar Gross', 'Max Theaters',
       'Total Gross', 'Release Date', 'Distributor', 'URL', 'Budget', 'Genres',
       'MPAA', 'Rating', 'Votes', '0', '1', '2', '3', '4', '5', '6', '7', '8',
       '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20',
       '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32',
       '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44',
       '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56',
       '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68',
       '69', '70', '71', '72', '73', '74'],
      dtype='object')

In [4]:
# drop all rows if 1st weekly gross is not recorded:
print(df_raw.shape)
df_raw = df_raw[df_raw['0'] != '0']
print(df_raw.shape)
display(df_raw.head(10))

(3422, 88)
(1121, 88)


,Unnamed: 0,Release,Calendar Gross,Max Theaters,Total Gross,Release Date,Distributor,URL,Budget,Genres,...,65,66,67,68,69,70,71,72,73,74
0,tt0021884,Gods and Monsters,"$4,784,214",149,"$6,451,628",Nov 6,Lionsgate\n\n,https://www.boxofficemojo.com/release/rl310932...,10000000,Biography Drama,...,"$2,526","$2,424","$2,809","$2,181","$1,916","$2,145","$3,028","$3,128","$3,230","$3,284"
77,tt0084805,Tootsie,"$146,259,321","1,222","$177,200,000",Dec 17,Columbia Pictures\n\n,https://www.boxofficemojo.com/release/rl645367...,21000000,Comedy Drama Romance,...,"$9,490","$7,838","$6,465","$5,635","$7,525","$5,470","$4,626","$4,301","$4,161","$4,053"
84,tt0086200,Risky Business,"$63,541,777","1,137","$63,541,777",Aug 5,Warner Bros.\n\n,https://www.boxofficemojo.com/release/rl40666625/,6200000,Comedy Crime Drama Romance,...,"$3,950","$3,346","$2,744","$2,491","$2,178","$1,885","$1,788","$1,359","$1,488","$1,330"
90,tt0086425,Terms of Endearment,"$34,003,216","1,259","$108,423,489",Nov 23,Paramount Pictures\n\n,https://www.boxofficemojo.com/release/rl324570...,8000000,Comedy Drama,...,"$8,998","$11,175","$8,294","$6,577","$4,830","$4,987","$4,461","$3,579","$4,433","$2,965"
91,tt0086425,Terms of Endearment,"$74,420,269","1,259","$108,423,489",23-Nov,Paramount Pictures\n\n,https://www.boxofficemojo.com/release/rl324570...,8000000,Comedy Drama,...,"$8,998","$11,175","$8,294","$6,577","$4,830","$4,987","$4,461","$3,579","$4,433","$2,965"
98,tt0087277,Footloose,"$80,035,402","1,384","$80,035,402",17-Feb,Paramount Pictures\n\n,https://www.boxofficemojo.com/release/rl200189...,8200000,Drama Music Romance,...,"$4,252","$3,916","$3,004","$3,588","$3,271","$2,666","$2,327","$1,945","$2,292","$2,570"
104,tt0087469,Indiana Jones and the Temple of Doom,"$179,870,271","1,687","$179,870,271",23-May,Paramount Pictures\n\n,https://www.boxofficemojo.com/release/rl245547...,28000000,Action Adventure,...,"$7,355","$6,217","$5,942","$5,050","$4,650","$3,955","$3,726","$3,550","$3,608","$2,914"
107,tt0088763,Back to the Future,"$20,028,355","1,550","$210,609,762",Jul 3,Universal Pictures\n\n,https://www.boxofficemojo.com/release/rl297438...,19000000,Adventure Comedy Sci-Fi,...,"$9,619","$8,827","$7,925","$6,938","$6,877","$4,402","$3,698","$3,499","$3,525","$3,130"
124,tt0091763,Platoon,"$136,772,865","1,564","$138,530,565",Dec 19,Orion Pictures\n\n,https://www.boxofficemojo.com/release/rl344617...,6000000,Drama War,...,"$24,664","$20,262","$13,709","$13,198","$8,603","$6,623","$5,471","$4,150","$3,545","$4,006"
143,tt0094137,Three Men and a Baby,"$90,528,478","1,813","$167,780,960",Nov 25,Walt Disney Studios Motion Pictures\n\n,https://www.boxofficemojo.com/release/rl306108...,11000000,Comedy Drama Family,...,"$11,976","$8,761","$5,854","$5,825","$4,125","$3,539","$2,851","$3,616","$2,627","$2,261"



***


#### Assign movies a unique numerical ID

In [5]:
# SKIP
#movie_prefix = years[0]*10000
#
#df_new['MovieID'] = [movie_prefix+int(i) for i in df_raw[yr+' Rank']]
#print(df_new)

In [6]:
#df_raw.columns


***


#### Convert to numerical format: Gross box office values, # Theaters, Budget, and MPAA rating; fix datetime column

In [7]:
df_new = pd.DataFrame()
df_new[['Release','Release Date','Genres','URL']] = df_raw[['Release','Release Date','Genres','URL']]

df_new['Distributor'] = [d.replace('\n\n','') for d in df_raw['Distributor']]
#df_new['Distributor'] = df_raw['Distributor']

df_new['Calendar Gross'] = [Decimal(sub(r'[^\d.]', '', g)) for g in df_raw['Calendar Gross']]
df_new['Total Gross']    = [Decimal(sub(r'[^\d.]', '', g)) for g in df_raw['Total Gross']]
df_new['Max Theaters']   = [int(s.replace(',','')) for s in df_raw['Max Theaters']]

#df_new['Budget'] = [int(s.replace('$','').replace(',','')) for s in df_raw['Budget']]

temp_R = [rating.replace('NC-17','5').replace('R','4').replace('PG-13','3').replace('PG','2').replace('G','1') for rating in df_raw['MPAA']]
df_new['MPAA_numeric'] = [int(i) for i in temp_R]

##temp_DT = [datetime.strptime(date, '%b %d %Y') for date in df_raw['Release Date']]
##df_new['Release Date'] = [item.strftime('%d-%m-%Y') for item in temp_DT]

### Append unchanged columns to new dataframe:
##df_new[['Distributor','Genres']] = df_raw[['Distributor','Genres']]
##display(df_new)


***


In [8]:
# Insert the other feature columns:
df_new[['Rating','Votes','Budget']] = df_raw[['Rating','Votes','Budget']]
display(df_new.head())

,Release,Release Date,Genres,URL,Distributor,Calendar Gross,Total Gross,Max Theaters,MPAA_numeric,Rating,Votes,Budget
0,Gods and Monsters,Nov 6,Biography Drama,https://www.boxofficemojo.com/release/rl310932...,Lionsgate,4784214,6451628,149,4,7.8,63657,10000000
77,Tootsie,Dec 17,Comedy Drama Romance,https://www.boxofficemojo.com/release/rl645367...,Columbia Pictures,146259321,177200000,1222,0,7.4,96363,21000000
84,Risky Business,Aug 5,Comedy Crime Drama Romance,https://www.boxofficemojo.com/release/rl40666625/,Warner Bros.,63541777,63541777,1137,0,6.8,79517,6200000
90,Terms of Endearment,Nov 23,Comedy Drama,https://www.boxofficemojo.com/release/rl324570...,Paramount Pictures,34003216,108423489,1259,0,7.4,53999,8000000
91,Terms of Endearment,23-Nov,Comedy Drama,https://www.boxofficemojo.com/release/rl324570...,Paramount Pictures,74420269,108423489,1259,0,7.4,53999,8000000


In [9]:
# features: weekly gross (wkGROSS_)
for i in range(0,15):
    week_x_gross = 'wkGROSS_'+str(i+1)
    df_new[week_x_gross] = [Decimal(sub(r'[^\d.]', '', g)) for g in df_raw[str(i)]]
# ==============================

In [10]:
# features: weekly percent changes in gross (pctCHNG_)
##for i in range(15,30):
##    pct_chng = 'pctCHNG_'+str(i-14)
##    df_new[pct_chng] = [Decimal(sub(r'[^-\d.]', '', g.replace('-','0'))) for g in df_raw[str(i)]]
# ==============================


In [11]:
# features: current number of theatres showing the film (nTHEATERS_)
for i in range(30,45):
##for i in range(15,30):
    nTheaters = 'nTHEATERS_'+str(i-29)
    ##nTheaters = 'nTHEATERS_'+str(i-14)
    df_new[nTheaters]   = [int(s.replace(',','').replace('-','0')) for s in df_raw[str(i)]]
# ==============================


In [12]:
# features: weekly change in number of theaters (dTHEATERS_)
##for i in range(45,60):
##    dTheaters = 'dTHEATERS'+str(i-44)
##    df_new[dTheaters]   = [int(s.replace(',','')) for s in df_raw[str(i)]]
##    df_new[pTAG] = [Decimal(sub(r'[^\d.]', '', g.replace('-','0'))) for g in df_raw[str(i)]]
# ==============================

In [13]:
# features: Per Theater Average Gross (pTAG_)
for i in range(60,75):
##for i in range(30,45):
    pTAG = 'pTAG_'+str(i-59)
    ##pTAG = 'pTAG_'+str(i-29)
    df_new[pTAG] = [Decimal(sub(r'[^-\d.]', '', g.replace('-','0'))) for g in df_raw[str(i)]]

In [14]:
df_new.head()

,Release,Release Date,Genres,URL,Distributor,Calendar Gross,Total Gross,Max Theaters,MPAA_numeric,Rating,...,pTAG_6,pTAG_7,pTAG_8,pTAG_9,pTAG_10,pTAG_11,pTAG_12,pTAG_13,pTAG_14,pTAG_15
0,Gods and Monsters,Nov 6,Biography Drama,https://www.boxofficemojo.com/release/rl310932...,Lionsgate,4784214,6451628,149,4,7.8,...,2526,2424,2809,2181,1916,2145,3028,3128,3230,3284
77,Tootsie,Dec 17,Comedy Drama Romance,https://www.boxofficemojo.com/release/rl645367...,Columbia Pictures,146259321,177200000,1222,0,7.4,...,9490,7838,6465,5635,7525,5470,4626,4301,4161,4053
84,Risky Business,Aug 5,Comedy Crime Drama Romance,https://www.boxofficemojo.com/release/rl40666625/,Warner Bros.,63541777,63541777,1137,0,6.8,...,3950,3346,2744,2491,2178,1885,1788,1359,1488,1330
90,Terms of Endearment,Nov 23,Comedy Drama,https://www.boxofficemojo.com/release/rl324570...,Paramount Pictures,34003216,108423489,1259,0,7.4,...,8998,11175,8294,6577,4830,4987,4461,3579,4433,2965
91,Terms of Endearment,23-Nov,Comedy Drama,https://www.boxofficemojo.com/release/rl324570...,Paramount Pictures,74420269,108423489,1259,0,7.4,...,8998,11175,8294,6577,4830,4987,4461,3579,4433,2965


In [15]:
# Write cleaned dataframe out to file:
df_new.to_csv('11_input_for_regression.csv')

In [ ]:
##from datetime import datetime

#####df_raw['Release Date'] = [i+' '+yr for i in df_raw['Release Date']]

##a = [datetime.strptime(date, '%b %d %Y') for date in df_raw['Release Date']]
#print(date_obj.strftime('%d-%m-%Y'))

##a = [datetime.strptime(date, '%b %d %Y') for date in df_raw['Release Date']]
##b = [item.strftime('%d-%m-%Y') for item in a]
#print(a)
##print(b)

In [ ]:
# Append unchanged columns to new dataframe:
##df_new[['Distributor','Genres']] = df_raw[['Distributor','Genres']]
##display(df_new)